# Artificial Neural Network

### Importing the libraries /라이브러리 호출

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [36]:
tf.__version__

'2.12.0'

## Part 1 - Data Preprocessing

### Importing the dataset / 데이터셋 불러오기

In [37]:
dataset = pd.read_csv('/content/drive/MyDrive/유데미 강의/머신러닝 딥러닝(Codes and DataSets)/Part 8 - Deep Learning/Section 39 - Artificial Neural Networks (ANN)/Python/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [38]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [39]:
y

array([1, 0, 1, ..., 1, 1, 0])

### Encoding categorical data / category형 데이터 인코딩
- 결측치가 없는 데이터셋이므로 결측치, 이상치 처리는 skip

Label Encoding the "Gender" column / "Gender"컬럼 레이블 인코딩

In [40]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])
# 3번째 컬럼(성별)에 대한 레이블 인코딩

In [41]:
print(X)
# 인코딩 확인

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column / "Geography"(지역) 컬럼 원핫 인코딩

In [42]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
# 2번째 컬럼(지역)에 대한 원핫 인코딩

In [43]:
print(X)
# 인코딩 확인

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set / 훈련, 테스트 데이터로 분리

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

### Feature Scaling / 피쳐 스케일링 (딥러닝에서 필수)

In [45]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

# 모든 컬럼 스케일링
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN / 인공신경망 생성

In [46]:
# 인공신경망 변수 -> 클래스의 오브젝트로 생성
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer / 입력층, 은닉층 추가

In [47]:
# 얕은 수준의 인공신경망 구축 -> 은닉층 1

ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))
# -> 입력층 자동으로 생성
# units : 은닉층 뉴런의 갯수 -> 하이퍼파라미터, 조정하면서 학습
# activation : 활성화 함수 지정 'relu'

### Adding the second hidden layer

In [48]:
# 2번째 은닉층 추가 -> 첫번째 은닉층의 생성 방식과 똑같다.
# 하이퍼 파라미터 조정 가능
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the output layer

In [49]:
# 출력층 추가(마지막 단계) -> units : 이진분류이므로 1
#                          -> 만약 A,B,C 와같은 다중분류인 경우 3 입력
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))
# activation -> sigmoid : 0~1 사이의 값을 반환
# 고객이 은행에 남을지(0 or 1) 알려주는 것 보다 은행에 남을 확률을 아는 것이 활용성이 높으므로

## Part 3 - Training the ANN

### Compiling the ANN

In [50]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# optimizer = 'adam' : 학습방식 -> 확률적 경사하강법(adam) 사용
# loss = 'binary_crossentropy' : 이진분류인 경우 binary_crossentropy 사용
#   -> 3가지 이상의 분류인 경우 : categorical_crossentropy_loss
# metrics : 모델 성과지표 선택

### Training the ANN on the Training set

In [51]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 50)
# batch_size : 한번에 훈련 할 행 묶음 (일반적으로 32 사용)
# epochs : 신경망 왕복횟수(연전파 횟수) 지정
# -> 결과 : epoch 횟수가 늘어나면서 accuracy점수가 높아지지만 일정 구간 이후로는 값이 수렴
# -> 이번 모델의 경우 epoch는 20정도만 해도 될듯 함.

Epoch 1/50
250/250 [==============================] - 1s 2ms/step - loss: 0.5886 - accuracy: 0.7461
Epoch 2/50
250/250 [==============================] - 0s 2ms/step - loss: 0.4992 - accuracy: 0.7974
Epoch 3/50
250/250 [==============================] - 0s 2ms/step - loss: 0.4618 - accuracy: 0.7985
Epoch 4/50
250/250 [==============================] - 0s 2ms/step - loss: 0.4253 - accuracy: 0.8064
Epoch 5/50
250/250 [==============================] - 0s 2ms/step - loss: 0.4007 - accuracy: 0.8214
Epoch 6/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3851 - accuracy: 0.8331
Epoch 7/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3756 - accuracy: 0.8380
Epoch 8/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3687 - accuracy: 0.8397
Epoch 9/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3642 - accuracy: 0.8430
Epoch 10/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3603 - accuracy: 0.8490

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

In [57]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
# 1. 예측 메서드의 입력값은 무조건 2D 배열임을 명시! -> [[]] 쌍대괄호 사용

# 2. France, Male 값을 인코딩 된 값으로 입력해야 함
# -> France 원핫 인코딩 결과는 1,0,0 이므로 1,0,0을 맨 앞에 입력
# -> 더미 변수의 경우 항상 첫번째 열에 생성됨.

# 3. 데이터 전체 스케일링 -> standard scaler (sc 적용)
# -> fit_transform을 하는 경우 입력된 값으로 기준점이 초기화(fit) 후 transform됨
# -> transform만 해줄것!!

# 결과 : 0.02 은행을 떠날 확률이 아주 낮음.

1/1 [==============================] - 0s 122ms/step
[[0.02362294]]


In [58]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
# 임계값(고객 이탈 확률) 0.5설정
# -> 임계값 이상의 확률이 나오는 경우 True : 고객이탈

1/1 [==============================] - 0s 34ms/step
[[False]]


**Solution**

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [59]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix / 테스트 데이터 예측 결과 및 혼동행렬 생성

In [60]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
# 실제값(y_test), 예측값(y_pred)으로 혼동행렬 생성

print(cm)
accuracy_score(y_test, y_pred)
# 최종 정확도

# 결과 :
# [[1525] 60]
#  [207 208]]
# 고객이 은행을 떠난다는 잘못된 예측 : 60건
# 고객이 은행에 남는다는 잘못된 예측 : 207건
# 정확도 : 약 0.87

[[1525   60]
 [ 207  208]]


0.8665